In [ ]:
# -----********************-----

# Created Time: 2024/09/06

# Author: Tara Liu, Yiyi He

### Use Case

# This notebook downloads ERA5-land data in GRIB format.
# Main function: download_era5_data(start_date, end_date, area, variables, folder_path)

# To use: 
# 1. add your API key
# 2. change to desired folder_path
# 3. input main function params

# -----********************-----

In [1]:
# Import libraries
import cdsapi
import os
import time
from datetime import datetime, timedelta

In [ ]:
# Create configuration file
def create_cdsapirc_in_home(content=""):
    """
    Creates or updates the .cdsapirc file in the user's home directory.
    """
    home_dir = os.path.expanduser("~")
    cdsapirc_path = os.path.join(home_dir, ".cdsapirc")

    try:
        with open(cdsapirc_path, 'w') as f:
            f.write(content)
        print(f"File '{cdsapirc_path}' created/updated successfully.")
    except Exception as e:
        print(f"Error creating/updating '{cdsapirc_path}': {e}")


In [ ]:
# Populate configuration file with url and api key
if __name__ == "__main__":
    config_content = """
[cdsapi]
url: https://cds.climate.copernicus.eu/api
key: 034022d1-0a8d-404b-86c3-b14a01b7c1bd
"""
    create_cdsapirc_in_home(config_content)

File '/Users/yiyi/.cdsapirc' created/updated successfully.


In [8]:
# Initialize 
c = cdsapi.Client()

# Define the folder path where the GRIB files will be saved
folder_path = '/Users/yiyi/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology(2)/Research/Energy_resilience/01_data/raw/2025'

# Ensure the folder exists (create if necessary)
os.makedirs(folder_path, exist_ok=True)

def get_existing_files(folder):
    """Get a set of filenames that already exist in the folder, and return them with their date and time."""
    existing_files = set()
    for filename in os.listdir(folder):
        if filename.endswith('.grib'):
            # Extract the date and time from the filename (assuming format: era5_land_YYYYMMDD_HHMM.grib)
            date_part = filename.split('_')[2]  # Extract YYYYMMDD part
            time_part = filename.split('_')[3].replace('.grib', '')  # Extract HHMM part
            existing_files.add(f"{date_part}_{time_part}")
    return existing_files

def print_existing_files_range(existing_files):
    """Print the date range of the existing files."""
    if not existing_files:
        print("No existing files found.")
        return
    dates = sorted([datetime.strptime(f"{f[:8]} {f[9:]}00", "%Y%m%d %H%M%S") for f in existing_files])
    print(f"Existing files cover from {dates[0]} to {dates[-1]}")

def download_era5_data(start_date, end_date, area, variables, folder_path):
    """
    
    Downloads ERA5 data for a specified date range and all hours of the day.
    Skips downloading if the file already exists.

    params:

    start_date: datetime object, the start date of the range
    end_date: datetime object, the end date of the range
    area: list of floats, the bounding box coordinates [N, W, S, E]
    variables: str, the variables to download
    folder_path: str, the folder path to save the GRIB files


    """
    # Get existing files
    existing_files = get_existing_files(folder_path)
    print_existing_files_range(existing_files)

    # Loop through each day in the range
    current_date = start_date
    while current_date <= end_date:
        # Extract the year, month, and day
        year = current_date.strftime('%Y')
        month = current_date.strftime('%m')
        day = current_date.strftime('%d')

        # Loop through all hours (00:00 to 23:00)
        for hour in range(24):
            # Format the hour as two digits
            hour_str = f"{hour:02d}:00"
            hour_for_filename = f"{hour:02d}00"

            # Filename for the GRIB file with the date and hour in the name
            filename = os.path.join(folder_path, f"era5_land_{year}{month}{day}_{hour_for_filename}.grib")

            # Check if the file already exists
            file_key = f"{year}{month}{day}_{hour_for_filename}"
            if file_key in existing_files:
                print(f"File for {year}-{month}-{day} {hour_str} already exists. Skipping download.")
                continue

            # Make the API call to retrieve data for the specific day and hour
            c.retrieve(
                'reanalysis-era5-land',
                {
                    'variable': variables,
                    'year': year,
                    'month': month,
                    'day': day,
                    'time': hour_str,  # Request for this specific hour
                    'area': area,
                    'format': 'grib',
                },
                filename
            )

            # Print a message when the file is saved
            print(f"Downloaded data for {year}-{month}-{day} {hour_str} and saved as {filename}")

        # Move to the next day
        current_date += timedelta(days=1)



In [ ]:
# Specify start and end date, bounding geometry, and variables to download
start_date = datetime(2025, 1, 1)
end_date = datetime(2025, 12, 31)
area = [31.0954, 72.4750, 12.4319, 94.802]  # North, West, South, East bounding box
variables = ['2m_temperature', '10m_u_component_of_wind', '10m_v_component_of_wind', 'total_precipitation']

# Run the function
download_era5_data(start_date, end_date, area, variables, folder_path)


No existing files found.


2025-07-05 06:45:37,496 INFO Request ID is ff109763-bccd-43f1-9eb6-1bf8fc373c59
2025-07-05 06:45:37,639 INFO status has been updated to accepted
Recovering from HTTP error [500 Internal Server Error], attemps 1 of 500
Retrying in 120 seconds
2025-07-05 06:47:39,719 INFO status has been updated to successful


Downloaded data for 2025-01-01 00:00 and saved as /Users/yiyi/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology(2)/Research/Energy_resilience/01_data/raw/2025/era5_land_20250101_0000.grib


2025-07-05 06:47:41,645 INFO Request ID is 51acebd2-6172-47f3-9d69-7d4913b629ba
Recovering from HTTP error [500 Internal Server Error], attemps 1 of 500
Retrying in 120 seconds
Recovering from HTTP error [500 Internal Server Error], attemps 2 of 500
Retrying in 120 seconds
2025-07-05 06:51:42,772 INFO status has been updated to successful


Downloaded data for 2025-01-01 01:00 and saved as /Users/yiyi/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology(2)/Research/Energy_resilience/01_data/raw/2025/era5_land_20250101_0100.grib


...

Downloaded data for 2025-07-01 11:00 and saved as /Users/yiyi/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology(2)/Research/Energy_resilience/01_data/raw/2025/era5_land_20250701_1100.grib


2025-07-06 08:42:36,081 INFO Request ID is e5d18387-3e11-4992-aae9-677e82b13d6d
2025-07-06 08:42:36,213 INFO status has been updated to accepted
2025-07-06 08:42:43,767 INFO status has been updated to running
2025-07-06 08:42:48,965 INFO status has been updated to successful
2025-07-06 08:42:50,748 INFO Request ID is 9c22bd83-b79d-4214-abc0-a718151da3ab        


Downloaded data for 2025-07-01 12:00 and saved as /Users/yiyi/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology(2)/Research/Energy_resilience/01_data/raw/2025/era5_land_20250701_1200.grib


2025-07-06 08:42:50,888 INFO status has been updated to accepted
2025-07-06 08:42:58,435 INFO status has been updated to failed


HTTPError: 400 Client Error: Bad Request for url: https://cds.climate.copernicus.eu/api/retrieve/v1/jobs/9c22bd83-b79d-4214-abc0-a718151da3ab/results
The job has failed.
None of the data you have requested is available yet, please revise the period requested. The latest date available for this dataset is: 2025-07-01 12:00
None of the data you have requested is available yet, please revise the period requested. The latest date available for this dataset is: 2025-07-01 12:00
The job failed with: MultiAdaptorNoDataError